In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install fastapi

In [2]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
from keras.models import load_model

from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

data = pd.read_csv('/content/drive/MyDrive/Dataset.csv')
col_list = data.columns.values.tolist()

@app.get('/home/{name}')
def home(name:str):
    return {'Welcome! ' + name}

@app.get('/modelpredictions/{modelInput}')
def get_predictions(modelInput:str):

    splitInputs = modelInput.split('-')

    modelInputs = []

    for val in splitInputs:
        modelInputs.append(float(val))

    modelInputs = np.array(modelInputs)

    minvals = [1,1,0,1,1,1,1,1,1,1]
    maxvals = [1000,3,1,3,3,5,3,3,100,3]

    for i in range(len(modelInputs)):
        modelInputs[i] = (modelInputs[i]-minvals[i])/(maxvals[i]-minvals[i])

    model_secure = load_model('/content/drive/MyDrive/Federated_models/Security.h5')
    model_scale = load_model('/content/drive/MyDrive/Federated_models/Scalability.h5')
    model_ease= load_model('/content/drive/MyDrive/Federated_models/Ease Of Use.h5')
    model_latency = load_model('/content/drive/MyDrive/Federated_models/Latency.h5')

    outmin = [1,1,1,1]
    outmax = [3,3,5,5]

    modelInputs = modelInputs.reshape((1,10))

    secure = model_secure.predict(modelInputs)[0][0]
    scale = model_scale.predict(modelInputs)[0][0]
    ease = model_ease.predict(modelInputs)[0][0]
    latency = model_latency.predict(modelInputs)[0][0]

    secure = round(secure * (outmax[0]-outmin[0]) + outmin[0])
    scale = round(scale * (outmax[1]-outmin[1]) + outmin[1])
    ease = round(ease * (outmax[2]-outmin[2]) + outmin[2])
    latency = round(latency * (outmax[3]-outmin[3]) + outmin[3])

    if secure<0:
        secure = 0
    if secure>3:
        secure = 3

    if scale<0:
        scale = 0
    if scale>3:
        scale = 3

    if ease<0:
        ease = 0
    if ease>5:
        ease = 5

    if latency<0:
        latency = 0
    if latency>5:
        latency = 5

    mp1 = {1:"Low", 2:"Medium", 3:"High"}
    mp2 = {1:"Strongly Disagree", 2:"Disagree", 3:"Neutral", 4:"Agree", 5:"Strongly Agree"}
    mp3 = {1:"Very High", 2:"High", 3:"Medium", 4:"Low", 5:"Very Low"}

    preds = {}
    preds['secure'] = mp1[secure]
    preds['scale'] = mp1[scale]
    preds['ease'] = mp2[ease]
    preds['latency'] = mp3[latency]
    
    return preds


@app.get('/efficiencyproduct/{product_id}')
def efficiencyProducts(product_id:str):

    product_info = {}
    product_id = 'Service_ID : ' + product_id

    for i in range(data.shape[0]):

        if data.iloc[i,0] == product_id:
            for j in range(data.iloc[i,:].shape[0]):
                product_info[col_list[j]] = str (data.iloc[i,j])
            break
    
    
    return product_info


@app.get('/similarfeatures/{outputFeatures}')
def similarFeatures(outputFeatures:str):
    
    outputFeatures = outputFeatures.split('-')

    similar_products = {}

    for i in range(data.shape[0]):
        
        if data['Security'].iloc[i] == outputFeatures[0] and data['Scalability'].iloc[i] == outputFeatures[1] and data['Ease Of Use'].iloc[i] == outputFeatures[2] and data['Latency'].iloc[i] == outputFeatures[3]:

            product_info = {}

            for j in range(data.iloc[i,:].shape[0]):
                product_info[col_list[j]] = str (data.iloc[i,j])

            similar_products[data.iloc[i,0]] = product_info

    return similar_products

@app.get('/similarrating/{rating}')
def similarRating(rating:str):
    similar_products = {}
    buffer = 3
    rating = float(rating)

    for i in range(data.shape[0]):

        if abs(data['Score'].iloc[i] - rating) < buffer:
            
            product_info = {}

            for j in range(data.iloc[i,:].shape[0]):
                product_info[col_list[j]] = str (data.iloc[i,j])
            
            similar_products[data.iloc[i,0]] = product_info
    return similar_products

@app.get('/')
def home():
    return({"key":"hello world"})

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://6685-35-185-16-76.ngrok.io


In [3]:
'''
/model-predictions/1-1-0-1-1-1-1-1-1-1
/efficiency-product/5
/similar-features/Low-High-Agree-Low
/similar-rating/95.5
'''

'\n/model-predictions/1-1-0-1-1-1-1-1-1-1\n/efficiency-product/5\n/similar-features/Low-High-Agree-Low\n/similar-rating/95.5\n'

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [504]
INFO:     Waiting for application startup.


In [4]:
modelInput = '999-1-2-3-1-3-3-1-10-5'

splitInputs = modelInput.split('-')

modelInputs = []

for val in splitInputs:
    modelInputs.append(float(val))

modelInputs = np.array(modelInputs)
print(modelInputs)
minvals = [1,1,0,1,1,1,1,1,1,1]
maxvals = [1000,3,1,3,3,5,3,3,100,3]

for i in range(len(modelInputs)):
    modelInputs[i] = (modelInputs[i]-minvals[i])/(maxvals[i]-minvals[i])

model_secure = load_model('/content/drive/MyDrive/Federated_models/Security.h5')
model_scale = load_model('/content/drive/MyDrive/Federated_models/Scalability.h5')
model_ease= load_model('/content/drive/MyDrive/Federated_models/Ease Of Use.h5')
model_latency = load_model('/content/drive/MyDrive/Federated_models/Latency.h5')

outmin = [1,1,1,1]
outmax = [3,3,5,5]

modelInputs = modelInputs.reshape((1,10))

print(modelInputs.shape)

secure = model_secure.predict(modelInputs)[0][0]
scale = model_scale.predict(modelInputs)[0][0]
ease = model_ease.predict(modelInputs)[0][0]
latency = model_latency.predict(modelInputs)[0][0]

print(secure, scale, ease, latency)

secure = round(secure * (outmax[0]-outmin[0]) + outmin[0])
scale = round(scale * (outmax[1]-outmin[1]) + outmin[1])
ease = round(ease * (outmax[2]-outmin[2]) + outmin[2])
latency = round(latency * (outmax[3]-outmin[3]) + outmin[3])

print(secure, scale, ease, latency)

mp1 = {1:"Low", 2:"Medium", 3:"High"}
mp2 = {1:"Strongly Disagree", 2:"Disagree", 3:"Neutral", 4:"Agree", 5:"Strongly Agree"}
mp3 = {1:"Very High", 2:"High", 3:"Medium", 4:"Low", 5:"Very Low"}

if secure<0:
    secure = 0
if secure>3:
    secure = 3

if scale<0:
    scale = 0
if scale>3:
    scale = 3

if ease<0:
    ease = 0
if ease>5:
    ease = 5

if latency<0:
    latency = 0
if latency>5:
    latency = 5


preds = {}
preds['secure'] = mp1[secure]
preds['scale'] = mp1[scale]
preds['ease'] = mp2[ease]
preds['latency'] = mp3[latency]
print(preds)

[999.   1.   2.   3.   1.   3.   3.   1.  10.   5.]
(1, 10)
0.119598374 0.39681065 1.1432859 0.46484438
1 2 6 3
{'secure': 'Low', 'scale': 'Medium', 'ease': 'Strongly Agree', 'latency': 'Medium'}


In [5]:
product_info = {}
product_id = '1'

product_id = 'Service_ID : ' + product_id
for i in range(data.shape[0]):

    if data.iloc[i,0] == product_id:
        for j in range(data.iloc[i,:].shape[0]):
            product_info[col_list[j]] = data.iloc[i,j]
        break


print(product_info)

{'Service Id': 'Service_ID : 1', 'Multi-Tenancy model': 789, 'Automated Provisioning': 'Low', 'Single Sign On': 'No', 'Subscription Based Billing': 'Medium', 'High Availability': 'Medium', 'Elastic Infrastructure': 'Neutral', 'Data Security': 'Low', 'Application Security': 'Medium', 'Rate Limiting/QoS': 74, 'Audit': 'Medium', 'Security': 'Low', 'Scalability': 'High', 'Ease Of Use': 'Neutral', 'Latency': 'Medium', 'Score': 59.375}


INFO:     Started server process [504]
INFO:     Waiting for application startup.


In [6]:
def similarFeatures(outputFeatures:str):
    
    outputFeatures = outputFeatures.split('-')

    similar_products = {}

    for i in range(data.shape[0]):
        
        if data['Security'].iloc[i] == outputFeatures[0] and data['Scalability'].iloc[i] == outputFeatures[1] and data['Ease Of Use'].iloc[i] == outputFeatures[2] and data['Latency'].iloc[i] == outputFeatures[3]:

            product_info = {}

            for j in range(data.iloc[i,:].shape[0]):
                product_info[col_list[j]] = data.iloc[i,j]

            similar_products[data.iloc[i,0]] = product_info

    return similar_products

outputFeatures = "Low-High-Agree-Low"

similar_products = similarFeatures(outputFeatures)
print(len(similar_products))
print(similar_products)

372
{'Service_ID : 18': {'Service Id': 'Service_ID : 18', 'Multi-Tenancy model': 808, 'Automated Provisioning': 'Medium', 'Single Sign On': 'No', 'Subscription Based Billing': 'High', 'High Availability': 'High', 'Elastic Infrastructure': 'Strongly Agree', 'Data Security': 'Medium', 'Application Security': 'Low', 'Rate Limiting/QoS': 46, 'Audit': 'Medium', 'Security': 'Low', 'Scalability': 'High', 'Ease Of Use': 'Agree', 'Latency': 'Low', 'Score': 71.875}, 'Service_ID : 24': {'Service Id': 'Service_ID : 24', 'Multi-Tenancy model': 855, 'Automated Provisioning': 'Medium', 'Single Sign On': 'Yes', 'Subscription Based Billing': 'Medium', 'High Availability': 'Low', 'Elastic Infrastructure': 'Neutral', 'Data Security': 'Low', 'Application Security': 'Low', 'Rate Limiting/QoS': 72, 'Audit': 'Medium', 'Security': 'Low', 'Scalability': 'High', 'Ease Of Use': 'Agree', 'Latency': 'Low', 'Score': 71.875}, 'Service_ID : 169': {'Service Id': 'Service_ID : 169', 'Multi-Tenancy model': 713, 'Automat

In [7]:
def similarRating(rating):

    similar_products = {}
    buffer = 3
    rating = float(rating)

    for i in range(data.shape[0]):

        if abs(data['Score'].iloc[i] - rating) < buffer:
            
            product_info = {}

            for j in range(data.iloc[i,:].shape[0]):
                product_info[col_list[j]] = data.iloc[i,j]
            
            similar_products[data.iloc[i,0]] = product_info

    return similar_products

rating = '95.5'

similar_products = similarRating(rating)

print(similar_products)

for i in similar_products:
    print(similar_products[i])

{'Service_ID : 87': {'Service Id': 'Service_ID : 87', 'Multi-Tenancy model': 197, 'Automated Provisioning': 'High', 'Single Sign On': 'Yes', 'Subscription Based Billing': 'High', 'High Availability': 'High', 'Elastic Infrastructure': 'Disagree', 'Data Security': 'High', 'Application Security': 'Medium', 'Rate Limiting/QoS': 96, 'Audit': 'Medium', 'Security': 'High', 'Scalability': 'Medium', 'Ease Of Use': 'Strongly Agree', 'Latency': 'Very Low', 'Score': 95.3125}, 'Service_ID : 96': {'Service Id': 'Service_ID : 96', 'Multi-Tenancy model': 724, 'Automated Provisioning': 'Medium', 'Single Sign On': 'Yes', 'Subscription Based Billing': 'High', 'High Availability': 'Medium', 'Elastic Infrastructure': 'Neutral', 'Data Security': 'High', 'Application Security': 'High', 'Rate Limiting/QoS': 98, 'Audit': 'Low', 'Security': 'High', 'Scalability': 'High', 'Ease Of Use': 'Agree', 'Latency': 'Very Low', 'Score': 93.75}, 'Service_ID : 129': {'Service Id': 'Service_ID : 129', 'Multi-Tenancy model': 